In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thelu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens
 
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	# load doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# update counts
	vocab.update(tokens)

# load all docs in a directory
def process_docs(directory, vocab):
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip files that do not have the right extension
		if not filename.endswith(".txt"):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# add doc to vocab
		add_doc_to_vocab(path, vocab)

# save list to file
def save_list(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()
 
# define vocab
vocab_pos = Counter()
vocab_neg = Counter()
# add all docs to vocab
process_docs('neg', vocab_neg)
process_docs('pos', vocab_pos)
# print the size of the vocab
print(len(vocab_neg))
print(len(vocab_pos))
# print the top words in the vocab
print(vocab_neg.most_common(50))
print(vocab_pos.most_common(50))
# keep tokens with > 5 occurrence in a dictionary
min_occurane = 5
tokensn = {k: str(c) for k,c in vocab_neg.items() if c >= min_occurane}
tokensp = {k: str(c) for k,c in vocab_pos.items() if c >= min_occurane}
print(len(tokensn))
print(len(tokensn))
# save tokens to a vocabulary file
save_list(tokensn, 'neg_occurance.txt')
save_list(tokensp, 'pos_occurance.txt')

30412
32487
[('film', 3600), ('movie', 2717), ('one', 2331), ('like', 1644), ('even', 1219), ('good', 1008), ('time', 994), ('would', 947), ('get', 930), ('bad', 923), ('much', 905), ('films', 828), ('characters', 810), ('story', 805), ('character', 799), ('two', 787), ('plot', 779), ('make', 731), ('first', 723), ('really', 715), ('see', 701), ('could', 694), ('way', 688), ('also', 685), ('little', 662), ('well', 646), ('dont', 602), ('movies', 602), ('scene', 593), ('people', 589), ('doesnt', 588), ('know', 586), ('never', 561), ('scenes', 558), ('action', 558), ('theres', 529), ('hes', 521), ('director', 515), ('new', 498), ('man', 494), ('another', 491), ('made', 488), ('end', 485), ('better', 475), ('something', 473), ('go', 468), ('big', 458), ('seems', 448), ('best', 447), ('isnt', 444)]
[('film', 4383), ('one', 2615), ('movie', 2109), ('like', 1557), ('story', 1102), ('good', 1072), ('also', 1072), ('time', 1047), ('films', 1045), ('even', 1043), ('characters', 925), ('much', 9

In [29]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords


# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
	# load the doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_doc(doc)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	return ' '.join(tokens)
 
# load all docs in a directory
def process_docs(directory, vocab):
	lines = list()
	# walk through all files in the folder
	for filename in listdir(directory):
		# skip files that do not have the right extension
		if not filename.endswith(".txt"):
			continue
		# create the full path of the file to open
		path = directory + '/' + filename
		# load and clean the doc
		line = doc_to_line(path, vocab)
		# add to list
		lines.append(line)
	return lines
 
# load vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# prepare negative reviews
negative_lines = process_docs('neg', vocab)
save_list(negative_lines, 'negative.txt')
# prepare positive reviews
positive_lines = process_docs('pos', vocab)
save_list(positive_lines, 'positive.txt')

In [34]:
neg_pvalue = {}
pos_pvalue = {}

# p values for all possible tokens in negative reviews
for token in tokensn:
	if token in tokensp:
		neg_pvalue[token] = float(tokensn[token]) / float(tokensn[token] + tokensp[token])

# p values for all possible tokens in positive reviews
for token in tokensp:
	if token in tokensn:
		pos_pvalue[token] = float(tokensp[token]) / float(tokensp[token] + tokensn[token])

print(neg_pvalue)
print(pos_pvalue)

{'plot': 0.0009993470248579554, 'two': 0.000998913507036819, 'teen': 0.009927007299270074, 'couples': 0.009907529722589167, 'go': 0.0009989668804057342, 'church': 0.009834368530020704, 'party': 0.009890258772524049, 'drink': 0.009913258983890954, 'drive': 0.009942920272509667, 'get': 0.0009991469648493651, 'accident': 0.009844134536505332, 'one': 9.998878289715675e-05, 'guys': 0.0009991948933233653, 'dies': 0.009892284018465597, 'girlfriend': 0.00990767845079937, 'continues': 0.009831460674157303, 'see': 0.000998780376601468, 'life': 0.0009979723017323893, 'nightmares': 0.09433962264150944, 'whats': 0.009940357852882704, 'deal': 0.0009984366583901522, 'watch': 0.0009989138793534014, 'movie': 9.999223836471435e-05, 'find': 0.0009988388498370644, 'critique': 0.009907997169143666, 'generation': 0.00988593155893536, 'touches': 0.009759271307742356, 'cool': 0.009930758017492712, 'idea': 0.0009992240068882677, 'presents': 0.00980392156862745, 'bad': 0.0009996480112636397, 'package': 0.009893